<a href="https://colab.research.google.com/github/pawan-cpu/STTP-Intel-Edge/blob/main/keras_resnet50_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Conversion to ONNX from keras model using tf2onnx python api 

In [ ]:
!pip install tensorflow tf2onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 442 kB 23.5 MB/s 
     |████████████████████████████████| 13.1 MB 64.3 MB/s 


In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 38.4 MB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import onnxruntime

img_path = '/content/semi-semitrailer-truck-tractor-highway.webp'

img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
x.shape

(1, 224, 224, 3)

### Run the keras model

In [ ]:
model = ResNet50(weights='imagenet')

preds = model.predict(x)
print('Keras Predicted:', decode_predictions(preds, top=3)[0])
model.save(os.path.join("/tmp", model.name))

1/1 [==============================] - 1s 815ms/step


Keras Predicted: [('n04467665', 'trailer_truck', 0.9776586), ('n03393912', 'freight_car', 0.012864678), ('n03796401', 'moving_van', 0.007267732)]


### Convert to ONNX using the Python API

In [ ]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = model.name + ".onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

### Run the ONNX model

In [ ]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": x})

print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])


ONNX Predicted: [('n04467665', 'trailer_truck', 0.9776586), ('n03393912', 'freight_car', 0.012864678), ('n03796401', 'moving_van', 0.0072677047)]


### Convert to ONNX using the command line

In [ ]:
!python -m tf2onnx.convert --opset 13 \
    --saved-model {os.path.join("/tmp", model.name)} \
    --output  {os.path.join("/tmp", model.name + ".onnx")}

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-11-11 08:30:45,490 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-11-11 08:30:50,804 - INFO - Signatures found in model: [serving_default].
2022-11-11 08:30:50,804 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-11-11 08:30:50,805 - INFO - Output names: ['predictions']
2022-11-11 08:30:54,927 - INFO - Using tensorflow=2.9.2, onnx=1.12.0, tf2onnx=1.13.0/2c1db5
2022-11-11 08:30:54,927 - INFO - Using opset <onnx, 13>
2022-11-11 08:31:00,433 - INFO - Computed 0 values for constant folding
2022-11-11 08:31:04,199 - INFO - Optimizing ONNX model
2022-11-11 08:31:05,359 - INFO - After optimization: Add -1 (18->17), BatchNormalization -53 (53->0), Const -161 (271->110), GlobalAveragePool 